<a href="https://colab.research.google.com/github/piyush1795/Natural-Language-Processing-in-TensorFlow/blob/master/Sentiment_Analysis_IMDB_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)


!pip install -q tensorflow-datasets

2.0.0-beta0


In [0]:
import tensorflow_datasets as tfds
imdb,info = tfds.load("imdb_reviews", with_info=True,as_supervised=True)

In [0]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [5]:
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

b this is the most depressing film i have ever seen i first saw it as a child and even thinking about it now really <OOV> me i know it was set in a time when life was hard and i know these people were poor and the <OOV> were vital yes i get all that what i find hard to take is i can't remember one single light moment in the entire film maybe it was true to life i don't know i'm quite sure the acting was top notch and the direction and quality of filming etc etc was wonderful and i know that every film can't have a happy ending but as a family film it is
b"This is the most depressing film I have ever seen. I first saw it as a child and even thinking about it now really upsets me. I know it was set in a time when life was hard and I know these people were poor and the crops were vital. Yes, I get all that. What I find hard to take is I can't remember one single light moment in the entire film. Maybe it was true to life, I don't know. I'm quite sure the acting was top notch and the direct

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [10]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [11]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs,validation_data=(testing_padded,testing_labels_final))

W0625 20:01:54.353809 140552242202496 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 4s 169us/sample - loss: 0.5108 - accuracy: 0.7264 - val_loss: 0.3508 - val_accuracy: 0.8461
Epoch 2/10
25000/25000 [==============================] - 4s 149us/sample - loss: 0.2481 - accuracy: 0.9034 - val_loss: 0.3644 - val_accuracy: 0.8394
Epoch 3/10
25000/25000 [==============================] - 4s 151us/sample - loss: 0.1029 - accuracy: 0.9729 - val_loss: 0.4339 - val_accuracy: 0.8312
Epoch 4/10
25000/25000 [==============================] - 4s 151us/sample - loss: 0.0271 - accuracy: 0.9962 - val_loss: 0.5194 - val_accuracy: 0.8243
Epoch 5/10
25000/25000 [==============================] - 4s 149us/sample - loss: 0.0070 - accuracy: 0.9994 - val_loss: 0.5863 - val_accuracy: 0.8269
Epoch 6/10
25000/25000 [==============================] - 4s 147us/sample - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.6369 - val_accuracy: 0.8268
Epoch 7/10
25000/25000 [==========================

In [12]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [0]:
import io

out_v = io.open('vecs.tsv','w',encoding='utf-8')
out_m = io.open('meta.tsv','w',encoding='utf-8')
for word_num in range(1,vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [15]:
sentence = 'I really think this is amazing. honest.'
sequence = tokenizer.texts_to_sequences(sentence)
print(sentence)

I really think this is amazing. honest.
